# COW(Copy-on-Write) 동작 원리 실습

이 노트북에서는 Iceberg의 **Copy-on-Write(COW)** 전략이 실제로 어떻게 동작하는지 파일 수준에서 관찰합니다.

## COW(Copy-on-Write)란?

COW는 Iceberg의 **기본 쓰기 전략**입니다.

### 핵심 원리
- 데이터 파일의 **단 한 행**이라도 업데이트/삭제되면, 해당 데이터 파일 **전체를 다시 씁니다**
- 기존 파일은 그대로 두고, 변경된 내용을 반영한 **새 파일**을 생성합니다

### 장점
- **읽기에 최적화**: 삭제 파일(delete file) 없이 데이터 파일만 읽으면 됩니다
- 읽기 시 추가적인 병합(merge) 작업이 필요 없습니다

### 단점
- **행 수준 업데이트/삭제가 느림**: 1건만 바꿔도 파일 전체를 재작성해야 합니다
- 쓰기 증폭(write amplification)이 발생합니다

```
예시: 1000행 파일에서 1행 UPDATE
┌─────────────────┐         ┌─────────────────┐
│  기존 파일        │         │  새 파일          │
│  (1000행)        │  ──→   │  (1000행, 1행 변경)│
│  ❌ 더 이상 참조 안됨│         │  ✅ 새 스냅샷이 참조│
└─────────────────┘         └─────────────────┘
```

## 환경 설정

In [1]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import (
    show_tree, snapshot_tree, diff_tree, count_files, total_size,
    show_metadata_hierarchy,
)

In [2]:
spark = create_spark_session()

TABLE_NAME = "demo.lab.cow_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/cow_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## COW 테이블 생성

모든 쓰기 모드(delete, update, merge)를 `copy-on-write`로 설정합니다.

In [3]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
""")

print(f"테이블 생성 완료: {TABLE_NAME}")

# 설정 확인
props = spark.sql(f"SHOW TBLPROPERTIES {TABLE_NAME}").collect()
for row in props:
    if 'write' in row['key']:
        print(f"  {row['key']} = {row['value']}")

테이블 생성 완료: demo.lab.cow_orders
  write.delete.mode = copy-on-write
  write.merge.mode = copy-on-write
  write.parquet.compression-codec = zstd
  write.update.mode = copy-on-write


---
## 실험 1: INSERT — 초기 데이터 적재

100건의 주문 데이터를 삽입하고, 파일 시스템 상태를 관찰합니다.

In [4]:
before_insert = snapshot_tree(TABLE_PATH)

# 100건 INSERT
orders = generate_orders(num_records=100, seed=42)
df = to_spark_df(spark, orders)
df.writeTo(TABLE_NAME).append()

after_insert = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("INSERT 후 변경 사항:")
print("=" * 60)
diff_tree(before_insert, after_insert)

INSERT 후 변경 사항:

[+] 추가된 파일 (6개):
    + data/order_date_month=2024-01/00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00001.parquet  (2.5 KB)
    + data/order_date_month=2024-02/00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00002.parquet  (2.5 KB)
    + data/order_date_month=2024-03/00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00003.parquet  (2.4 KB)
    + metadata/a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73-m0.avro  (7.3 KB)
    + metadata/snap-8931743094173598432-1-a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73.avro  (4.2 KB)
    + metadata/v2.metadata.json  (2.7 KB)

요약: +6 추가, -0 삭제, ~0 변경


In [5]:
print("INSERT 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")

INSERT 후 테이블 트리 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00001.parquet  (2.5 KB)
│   ├── order_date_month=2024-02/
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00002.parquet  (2.5 KB)
│   └── order_date_month=2024-03/
│       └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00003.parquet  (2.4 KB)
└── metadata/
    ├── a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73-m0.avro  (7.3 KB)
    ├── snap-8931743094173598432-1-a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73.avro  (4.2 KB)
    ├── v1.metadata.json  (1.6 KB)
    ├── v2.metadata.json  (2.7 KB)
    └── version-hint.text  (1 B)

파일 수: 3
총 크기: 23,667 bytes


In [6]:
# 데이터 확인
print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")
print("\n상태별 건수:")
spark.sql(f"SELECT status, COUNT(*) as cnt FROM {TABLE_NAME} GROUP BY status ORDER BY cnt DESC").show()

총 레코드 수: 100

상태별 건수:
+----------+---+
|    status|cnt|
+----------+---+
|processing| 24|
|   shipped| 22|
|   pending| 19|
| cancelled| 18|
| completed| 17|
+----------+---+



In [7]:
# 스냅샷 확인
print("현재 스냅샷:")
spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").show(truncate=False)

현재 스냅샷:
+----------------------+-------------------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at          |snapshot_id        |parent_id|operation|manifest_list                                                                                                                |summary                                                                                                                                                                                                                                                                                             |
+----------------------+

### 관찰 포인트 — INSERT

- INSERT는 COW/MOR에 관계없이 동일하게 동작합니다
- 파티션(months)별로 데이터 파일이 생성되었습니다
- 스냅샷이 1개 생성되었습니다

---
## 실험 2: UPDATE — COW의 핵심 동작 관찰

`status='cancelled'`인 주문을 `status='refunded'`로 변경합니다.  
COW에서는 변경된 행이 포함된 파티션의 데이터 파일이 **통째로 재작성**됩니다.

In [8]:
# UPDATE 대상 확인
cancelled_count = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE status = 'cancelled'").collect()[0][0]
print(f"UPDATE 대상 (status='cancelled'): {cancelled_count}건")
print("\n파티션별 분포:")
spark.sql(f"""
    SELECT month(order_date) as partition_month, COUNT(*) as cnt
    FROM {TABLE_NAME}
    WHERE status = 'cancelled'
    GROUP BY month(order_date)
    ORDER BY partition_month
""").show()

UPDATE 대상 (status='cancelled'): 18건

파티션별 분포:
+---------------+---+
|partition_month|cnt|
+---------------+---+
|              1|  5|
|              2|  6|
|              3|  7|
+---------------+---+



In [9]:
before_update = snapshot_tree(TABLE_PATH)

# UPDATE 실행
spark.sql(f"UPDATE {TABLE_NAME} SET status = 'refunded' WHERE status = 'cancelled'")

after_update = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("UPDATE 후 변경 사항 (COW):")
print("=" * 60)
diff_tree(before_update, after_update)

UPDATE 후 변경 사항 (COW):

[+] 추가된 파일 (7개):
    + data/order_date_month=2024-01/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet  (2.5 KB)
    + data/order_date_month=2024-02/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00002.parquet  (2.5 KB)
    + data/order_date_month=2024-03/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00003.parquet  (2.4 KB)
    + metadata/b41ba155-557d-4f22-9b3e-9e267ff97a5c-m0.avro  (7.3 KB)
    + metadata/b41ba155-557d-4f22-9b3e-9e267ff97a5c-m1.avro  (7.3 KB)
    + metadata/snap-8596805567379484441-1-b41ba155-557d-4f22-9b3e-9e267ff97a5c.avro  (4.2 KB)
    + metadata/v3.metadata.json  (3.8 KB)

요약: +7 추가, -0 삭제, ~0 변경


In [10]:
print("UPDATE 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")

UPDATE 후 테이블 트리 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet  (2.5 KB)
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00001.parquet  (2.5 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00002.parquet  (2.5 KB)
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00002.parquet  (2.5 KB)
│   └── order_date_month=2024-03/
│       ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00003.parquet  (2.4 KB)
│       └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00003.parquet  (2.4 KB)
└── metadata/
    ├── a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73-m0.avro  (7.3 KB)
    ├── b41ba155-557d-4f22-9b3e-9e267ff97a5c-m0.avro  (7.3 KB)
    ├── b41ba155-557d-4f22-9b3e-9e267ff97a5c-m1.avro  (7.3 KB)
    ├── snap-8596805567379484441-1-b41ba155-557d-4f22-9b3e-9e267ff97a5c.avro  (4.2 KB)
    ├── snap-8931743094173598432-1-a5a35bf0-b7da-4b86-ae10-8a16cc3a6f73.avro  (4.2 KB)
  

### Metadata 계층 구조로 보는 COW UPDATE

파일 시스템의 변화를 확인했으니, 이제 Iceberg의 **전체 메타데이터 계층**을 따라가며 UPDATE가 내부적으로 어떻게 기록되는지 확인합니다.

```
metadata.json   ← 현재 스냅샷 ID를 가리킴
│
└─▶ snap-*.avro   [Manifest List]  ← 이 스냅샷이 참조하는 Manifest 목록
    ├─▶ *-m0.avro  [Manifest File]  ← 새 파일 목록 (ADDED)
    │   └── *.parquet               ← 실제 데이터 파일
    └─▶ *-m1.avro  [Manifest File]  ← 기존 파일 상태 (DELETED)
        └── *.parquet               ← 더 이상 참조되지 않는 파일
```

각 파일의 상태: **ADDED**(이 스냅샷에서 추가) · **EXISTING**(이전에서 유지) · **DELETED**(이 스냅샷에서 제거)

In [11]:
# metadata.json → manifest list → manifest file → data file 전체 계층 확인
show_metadata_hierarchy(TABLE_PATH)

v3.metadata.json  (operation: overwrite)
│
└─▶ snap-8596805567379484441-1-b41ba155-557d-4f22-9b3e-9e267ff97a5c.avro  [Manifest List]
    ├─▶ b41ba155-557d-4f22-9b3e-9e267ff97a5c-m1.avro  [Manifest — DATA: 3 ADDED]
        │   ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-01/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet  (38행, ADDED)
        │   ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-02/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00002.parquet  (32행, ADDED)
        │   └── data/warehouse/lab/cow_orders/data/order_date_month=2024-03/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00003.parquet  (30행, ADDED)
    └─▶ b41ba155-557d-4f22-9b3e-9e267ff97a5c-m0.avro  [Manifest — DATA: 3 DELETED]
            ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-01/00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00001.parquet  (38행, DELETED)
            ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-02/00000-5-4e878d0d-1aae-4e48

In [12]:
# 파일 수준에서 비교: 기존 파일 vs 새 파일의 레코드 수
print("파일별 레코드 수 확인:")
spark.sql(f"""
    SELECT 
        regexp_replace(file_path, '^file:.*?/(data/)', '$1') as file_path, 
        record_count, 
        file_size_in_bytes
    FROM {TABLE_NAME}.files
    ORDER BY file_path
""").show(truncate=False)

파일별 레코드 수 확인:
+-----------------------------------------------------------------------------------------------------------------------+------------+------------------+
|file_path                                                                                                              |record_count|file_size_in_bytes|
+-----------------------------------------------------------------------------------------------------------------------+------------+------------------+
|data/warehouse/lab/cow_orders/data/order_date_month=2024-01/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet|38          |2546              |
|data/warehouse/lab/cow_orders/data/order_date_month=2024-02/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00002.parquet|32          |2520              |
|data/warehouse/lab/cow_orders/data/order_date_month=2024-03/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00003.parquet|30          |2461              |
+-------------------------------------------------------------

In [13]:
# 스냅샷 확인
print("현재 스냅샷:")
spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").show(truncate=False)

현재 스냅샷:
+-----------------------+-------------------+-------------------+---------+-----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                |summary                                                                                                                                                                                                                    

### 관찰 포인트 — UPDATE (COW)

- `cancelled` 상태의 주문이 포함된 **파티션의 데이터 파일이 통째로 재작성**되었습니다
- 새 데이터 파일이 추가되고, 기존 파일은 메타데이터에서 참조가 해제됩니다
- **Delete File은 생성되지 않았습니다** — 이것이 COW의 특징입니다
- 영향받지 않은 파티션의 파일은 그대로 유지됩니다
- 스냅샷이 1개 더 추가되었습니다 (총 2개)

---
## 실험 3: DELETE — 파일 재작성 패턴 재확인

`amount < 200`인 주문을 삭제합니다.  
DELETE도 UPDATE와 동일하게 파일 전체가 재작성됩니다.

In [14]:
# DELETE 대상 확인
delete_count = spark.sql(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE amount < 200").collect()[0][0]
print(f"DELETE 대상 (amount < 200): {delete_count}건")
print(f"DELETE 전 총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

DELETE 대상 (amount < 200): 13건
DELETE 전 총 레코드 수: 100


In [15]:
before_delete = snapshot_tree(TABLE_PATH)

# DELETE 실행
spark.sql(f"DELETE FROM {TABLE_NAME} WHERE amount < 200")

after_delete = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("DELETE 후 변경 사항 (COW):")
print("=" * 60)
diff_tree(before_delete, after_delete)

DELETE 후 변경 사항 (COW):

[+] 추가된 파일 (7개):
    + data/order_date_month=2024-01/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00001.parquet  (2.4 KB)
    + data/order_date_month=2024-02/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00002.parquet  (2.4 KB)
    + data/order_date_month=2024-03/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00003.parquet  (2.4 KB)
    + metadata/5cdde880-5fb5-42b5-88a6-174d185ed341-m0.avro  (7.3 KB)
    + metadata/5cdde880-5fb5-42b5-88a6-174d185ed341-m1.avro  (7.3 KB)
    + metadata/snap-3267856845968573308-1-5cdde880-5fb5-42b5-88a6-174d185ed341.avro  (4.2 KB)
    + metadata/v4.metadata.json  (4.8 KB)

요약: +7 추가, -0 삭제, ~0 변경


In [16]:
print("DELETE 후 테이블 트리 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

print(f"\n파일 수: {count_files(TABLE_PATH)}")
print(f"총 크기: {total_size(TABLE_PATH):,} bytes")
print(f"DELETE 후 총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

DELETE 후 테이블 트리 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet  (2.5 KB)
│   │   ├── 00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00001.parquet  (2.4 KB)
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00001.parquet  (2.5 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00002.parquet  (2.5 KB)
│   │   ├── 00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00002.parquet  (2.4 KB)
│   │   └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00002.parquet  (2.5 KB)
│   └── order_date_month=2024-03/
│       ├── 00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00003.parquet  (2.4 KB)
│       ├── 00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00003.parquet  (2.4 KB)
│       └── 00000-5-4e878d0d-1aae-4e48-a33b-750726599cef-00003.parquet  (2.4 KB)
└── metadata/
    ├── 5cdde880-5fb5-42b5-88a6-174d185ed341-m0.avro  (7.3 KB)
    ├── 5cdde880-5fb5-42b5-88a6-174d185ed341-m1.avro  (7

In [17]:
# DELETE 후에도 동일한 패턴: DELETED + ADDED
show_metadata_hierarchy(TABLE_PATH)

print("\n→ COW에서는 INSERT/UPDATE/DELETE 모두 '파일 전체 재작성' 방식입니다.")
print("  Manifest에서 기존 파일은 DELETED, 새 파일은 ADDED로 기록됩니다.")

v4.metadata.json  (operation: overwrite)
│
└─▶ snap-3267856845968573308-1-5cdde880-5fb5-42b5-88a6-174d185ed341.avro  [Manifest List]
    ├─▶ 5cdde880-5fb5-42b5-88a6-174d185ed341-m1.avro  [Manifest — DATA: 3 ADDED]
        │   ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-01/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00001.parquet  (32행, ADDED)
        │   ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-02/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00002.parquet  (28행, ADDED)
        │   └── data/warehouse/lab/cow_orders/data/order_date_month=2024-03/00000-22-fe7aa32e-536b-43fb-a3b7-7c62a7cf3334-00003.parquet  (27행, ADDED)
    └─▶ 5cdde880-5fb5-42b5-88a6-174d185ed341-m0.avro  [Manifest — DATA: 3 DELETED]
            ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-01/00000-14-7286ce55-ba45-436b-93be-e0cfe833abee-00001.parquet  (38행, DELETED)
            ├── data/warehouse/lab/cow_orders/data/order_date_month=2024-02/00000-14-7286ce55-ba45-43

In [18]:
# 스냅샷 확인
print("현재 스냅샷:")
spark.sql(f"SELECT * FROM {TABLE_NAME}.snapshots").show(truncate=False)

현재 스냅샷:
+-----------------------+-------------------+-------------------+---------+-----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                |summary                                                                                                                                                                                                                    

### 관찰 포인트 — DELETE (COW)

- DELETE도 UPDATE와 동일한 패턴: 영향받은 파티션의 **데이터 파일이 통째로 재작성**됩니다
- 삭제된 행을 제외한 나머지 행만 포함된 **새 파일**이 생성됩니다
- 역시 **Delete File은 생성되지 않았습니다**
- 스냅샷이 1개 더 추가되었습니다 (총 3개)

---
## 정리: COW(Copy-on-Write) 핵심 요약

| 항목 | 설명 |
|------|------|
| **쓰기 방식** | 변경된 행이 포함된 데이터 파일 전체를 새로 작성 |
| **Delete File** | 생성하지 않음 |
| **읽기 성능** | 최적 — 데이터 파일만 읽으면 됨 |
| **쓰기 성능** | 느림 — 1건 변경에도 파일 전체 재작성 |
| **적합한 워크로드** | 읽기 중심, 업데이트/삭제가 적은 경우 |

> **핵심**: COW에서는 **1건만 바꿔도 해당 파티션의 데이터 파일이 통째로 재작성**됩니다.  
> 다음 노트북에서 MOR(Merge-on-Read)이 이 문제를 어떻게 해결하는지 살펴보겠습니다.

In [19]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
